In [ ]:
import nidaqmx as ni
import numpy as np
from hardware.nidaq import sample_clock, analog_output_sweeper
from hardware.nidaq_finite_scanner import piezostage_controller_aom

In [2]:
device_name = 'dev2'
counter_name = 'ctr0'
ao_channels = ['ao0', 'ao1', 'ao2', 'ao3']

voltage_range = [
    [0., 1.],       # ao0
    [0., 1.],       # ao1
    [0., 1.],       # ao2
    [0., 1.],       # ao3
]

period = 0.01   # seconds
sample_rate = 1/period
DutyCycle = 0.9
frame_size = 10

clk = sample_clock(
    device_name, counter_name, samps_per_chan=frame_size
)
clk.prepare_task()

ao_sweep = analog_output_sweeper(
    device_name, ao_channels, voltage_range,
    clk, use_falling=True
)
ao_sweep.prepare_task()

from hardware.time_tagger_swabian import time_tagger_control
time_tagger = time_tagger_control("1740000JEJ", 1, 5, 8, ch_marker=2)

In [ ]:
params_dict = dict(
    x_range=(-100.0,100.0),
    y_range=(-100.0,100.0),
    z_range=(0,100.0),
    aom_range=(-10,10),
    invert_x=False,
    invert_y=False,
    invert_z=False,
    swap_xy=False,
)
stage = piezostage_controller_aom(
    ao_sweep, clk, time_tagger, **params_dict
)

(4, 1) (4, 1) (4, 1)
[[0.5]
 [0.5]
 [0.5]
 [0.5]]


In [7]:
stage.scanLine([
    np.linspace(0,50,6),
    np.ones(6)*0,
    np.ones(6)*50.,
    np.ones(6)*0,
],SecondsPerPoint=.01,add_aom=False)

(4, 1) (4, 1) (4, 6)
[[0.5  0.55 0.6  0.65 0.7  0.75]
 [0.5  0.5  0.5  0.5  0.5  0.5 ]
 [0.5  0.5  0.5  0.5  0.5  0.5 ]
 [0.5  0.5  0.5  0.5  0.5  0.5 ]]
(4, 1) (4, 1) (4, 1)
[[0.5]
 [0.5]
 [0.5]
 [0.5]]


array([810., 900., 720., 630., 990., 630.])

In [9]:
ao_sweep.clear_task()
clk.clear_task()

No task has been created.
No task has been created.


In [15]:
a = np.arange(12).reshape((4,3))
mask = [False, False, False, False]
a[mask] = np.ones((4,3))[mask]


In [5]:
ao_sweep.task.is_task_done()

True

In [7]:
ao_sweep.samps_per_chan

6

In [5]:
ao_sweep.task.timing.samp_timing_type #= ni.constants.SampleTimingType.ON_DEMAND
# clk.task.timing.samp_quant_samp_mode

<SampleTimingType.ON_DEMAND: 10390>